In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
# définition des classes Generator et Discriminator"
# Définir le générateur
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels, img_size):
        super().__init__()

        # Architecture du générateur
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, img_size * img_size * img_channels),
            nn.Tanh()
        )
        
        self.img_size = img_size
        self.img_channels = img_channels

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), self.img_channels, self.img_size, self.img_size)
        return x

# Définir le discriminateur
class Discriminator(nn.Module):
    def __init__(self, img_channels, img_size):
        super().__init__()

        # Architecture du discriminateur
        self.model = nn.Sequential(
            nn.Linear(img_size * img_size * img_channels, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

        self.img_size = img_size
        self.img_channels = img_channels

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.model(x)
        return x


In [5]:
#Initialisation des modèles, déclaration des fonctions de perte et des optimiseurs
# Définir les dimensions et autres paramètres
latent_dim = 100  # Dimension du vecteur de bruit
img_channels = 3  # Nombre de canaux de couleur dans les images
img_size = 64  # Taille des images

# Initialiser les modèles
generator = Generator(latent_dim, img_channels, img_size)
discriminator = Discriminator(img_channels, img_size)

# Fonction de perte
adversarial_loss = nn.BCELoss()

# Définir les optimiseurs
lr = 0.0002
beta1 = 0.5
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))


In [ ]:
#Configuration du DataLoader et définition des fonctions auxiliaires
# GPU Setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)

# Configuration du DataLoader
dataset_path = "C:/Users/DELL/Desktop/abstract_art_dataset.csv"  # Chemin vers database contenant les images
batch_size = 64

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = ImageFolder(dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Fonction pour générer du bruit
def generate_noise(batch_size, latent_dim):
    return torch.randn(batch_size, latent_dim, 1, 1, device=device)


In [ ]:
#Boucle d'entraînement
num_epochs = 50

for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Préparation des vraies images
        real_imgs = imgs.to(device)

        # Entraînement du Discriminateur
        optimizer_D.zero_grad()
        
        # Générer des images à partir du bruit pour l'entraînement du générateur
        z = generate_noise(imgs.size(0), latent_dim)
        fake_imgs = generator(z).detach()
        
        # Calcul de la perte pour les vraies et fausses images
        real_labels = torch.ones(imgs.size(0), 1, device=device)
        fake_labels = torch.zeros(imgs.size(0), 1, device=device)
        
        real_loss = adversarial_loss(discriminator(real_imgs), real_labels)
        fake_loss = adversarial_loss(discriminator(fake_imgs), fake_labels)
        
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Entraînement du Générateur
        optimizer_G.zero_grad()
        z = generate_noise(imgs.size(0), latent_dim)
        fake_imgs = generator(z)
        
        g_loss = adversarial_loss(discriminator(fake_imgs), real_labels)
        g_loss.backward()
        optimizer_G.step()

        if i % 100 == 0:
            print(
                f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]"
            )
